**1. Import dependencies**

In [1]:
import tensorflow as tf
import matplotlib.pyplot as plt
import numpy as np
import tensorflow.keras.preprocessing.image as Image
from tf_bodypix.api import download_model, load_model, BodyPixModelPaths
from pathlib import Path

2024-04-13 20:29:38.860632: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
output_path = Path('outputs')
input_path = Path('inputs')
images_path = input_path.glob('*')
# target_size = (480, 480)

**2. Init Constant**

In [3]:
body_parts = {
  'left_arm': ['left_upper_arm_front', 'left_upper_arm_back', 'left_lower_arm_front', 'left_lower_arm_back'],
  'right_arm': ['right_upper_arm_front', 'right_upper_arm_back', 'right_lower_arm_front', 'right_lower_arm_back'],
  'torso': ['torso_front', 'torso_back'],
  'left_leg': ['left_upper_leg_front', 'left_upper_leg_back', 'left_lower_leg_front', 'left_lower_leg_back'],
  'right_leg': ['right_upper_leg_front', 'right_upper_leg_back', 'right_lower_leg_front', 'right_lower_leg_back']
}

**3. Load model**

In [4]:
model_path = BodyPixModelPaths.RESNET50_FLOAT_STRIDE_16
downloaded_model = download_model(model_path)
bodypix_model = load_model(downloaded_model)

2024-04-13 20:29:49.278928: I tensorflow/core/grappler/devices.cc:75] Number of eligible GPUs (core count >= 8, compute capability >= 0.0): 0 (Note: TensorFlow was not compiled with CUDA or ROCm support)


**4. Detections**

In [5]:
def get_first_y(image_array):
  for y in range(image_array.shape[0]):
    for x in range(image_array.shape[1]):
      if not np.allclose(image_array[y][x], 0):
        return y
    
def get_y_axis(image_array):
  first_y = get_first_y(image_array)
  
  if first_y is None:
    return [0, 0, 0]
  
  for y in range(first_y, image_array.shape[0]):
    cnt = 0
    
    for x in range(image_array.shape[1]):
      if np.allclose(image_array[y][x], 0):
        cnt += 1
        
    if cnt == image_array.shape[1] or y == image_array.shape[0] - 1:
      last_y = y - 1
      height = abs(last_y - first_y)
      return [first_y, last_y, height]
  return [0, 0, 0]
    
def get_first_x(image_array):
  for x in range(image_array.shape[1]):
    for y in range(image_array.shape[0]):
      if not np.allclose(image_array[y][x], 0):
        return x
      
def get_x_axis(image_array):
  first_x = get_first_x(image_array)
  
  if first_x is None:
    return [0, 0, 0]

  for x in range(first_x, image_array.shape[1]):
    cnt = 0
    
    for y in range(image_array.shape[0]):
      if np.allclose(image_array[y][x], 0):
        cnt += 1
        
    if cnt == image_array.shape[0] or x == image_array.shape[1] - 1:
      last_x = x - 1
      width = abs(last_x - first_x)
      return [first_x, last_x, width]
  return [0, 0, 0]
    
def get_part_image(image_array, image):
  x_axis = get_x_axis(image_array=image_array)
  y_axis = get_y_axis(image_array=image_array)
  
  width = x_axis[2] if x_axis[2] != 0 else 10
  height = y_axis[2] if y_axis[2] != 0 else 10
  first_x, last_x = x_axis[0], x_axis[1]
  first_y, last_y = y_axis[0], y_axis[1]
  print(f'fx: {first_x}, lx: {last_x}, fy: {first_y}, ly: {last_y}')
  
  pixels = np.array([[[0, 0, 0]] * width] * height)
  image_data = np.array(image.getdata())
  
  for i in range (first_y, last_y):
    for j in range(first_x, last_x):
      if not np.allclose(image_array[i][j], 0):
        pixels[i - first_y][j - first_x] = image_data[image_array.shape[1] * i + j]
        
  return pixels

def save_part_mask_image(result, mask, image_name, threshold):
  for i, part in enumerate(body_parts):      
    part_mask = result.get_part_mask(mask=mask, part_names=body_parts[part])
    Image.save_img(
      f'{output_path}/{str(threshold)}/{image_name}/{part}.jpg',
      part_mask
  )
    
def save_colored_mask_image(result, mask, image_name, threshold):
  for i, part in enumerate(body_parts):      
    colored_part_mask = result.get_colored_part_mask(mask=mask, part_names=body_parts[part])
    Image.save_img(
      f'{output_path}/{str(threshold)}/{image_name}/{part}.jpg',
      colored_part_mask
  )
    
def save_part_image(image_name, image, threshold):
  image_parts = [[]] * 5

  for i, part in enumerate(body_parts):
    image_parts[i] = Image.load_img(f'{output_path}/{str(threshold)}/{image_name}/{part}.jpg')
    image_array = Image.img_to_array(image_parts[i])  
    
    pixels_data = get_part_image(image_array=image_array, image=image)
    
    Image.save_img(
      f'{output_path}/{str(threshold)}/{image_name}/{part}.jpg',
      pixels_data
    )
    
    print(i, part, pixels_data.shape)

In [ ]:
for image_path in images_path:
  # image = Image.load_img(image_path).resize(target_size)
  image = Image.load_img(image_path)
  image_array = Image.img_to_array(image)
  result = bodypix_model.predict_single(image_array)
  # for i in range(1, 11):
  #   threshold = i / 10
  threshold = 0.7
  mask = result.get_mask(threshold=threshold)
  
  image_name = Path(image_path.name).stem
  output_path.joinpath(str(threshold), image_name).mkdir(parents=True, exist_ok=True)
  print(f'Created output path {output_path}/{str(threshold)}/{image_name}')
  
  save_part_mask_image(result=result, mask=mask, image_name=image_name, threshold=threshold)
  # save_colored_mask_image(result=result, mask=mask, image_name=image_name, threshold=threshold)
  save_part_image(image_name=image_name, image=image, threshold=threshold)
  print(f'Saved images to output path {output_path}/{str(threshold)}')

**5. Try with removed background images**

In [6]:
output_path = Path('rmbg_outputs')
input_path = Path('rmbg_inputs')
images_path = input_path.glob('*')

In [7]:
for image_path in images_path:
  # image = Image.load_img(image_path).resize(target_size)
  image = Image.load_img(image_path)
  image_array = Image.img_to_array(image)
  result = bodypix_model.predict_single(image_array)
  # for i in range(1, 11):
  threshold = 0.7
  mask = result.get_mask(threshold=threshold)
  
  image_name = Path(image_path.name).stem
  output_path.joinpath(str(threshold), image_name).mkdir(parents=True, exist_ok=True)
  print(f'Created output path {output_path}/{str(threshold)}/{image_name}')
  
  save_part_mask_image(result=result, mask=mask, image_name=image_name, threshold=threshold)
  # save_colored_mask_image(result=result, mask=mask, image_name=image_name, threshold=threshold)
  save_part_image(image_name=image_name, image=image, threshold=threshold)
  print(f'Saved images to output path {output_path}/{str(threshold)}')

Created output path rmbg_outputs/0.7/15_26
fx: 448, lx: 535, fy: 272, ly: 543
0 left_arm (271, 87, 3)
fx: 184, lx: 255, fy: 280, ly: 551
1 right_arm (271, 71, 3)
fx: 224, lx: 487, fy: 232, ly: 663
2 torso (431, 263, 3)
fx: 360, lx: 495, fy: 600, ly: 1087
3 left_leg (487, 135, 3)
fx: 224, lx: 367, fy: 584, ly: 1095
4 right_leg (511, 143, 3)
Saved images to output path rmbg_outputs/0.7
Created output path rmbg_outputs/0.7/0_19
fx: 648, lx: 735, fy: 392, ly: 647
0 left_arm (255, 87, 3)
fx: 240, lx: 383, fy: 368, ly: 799
1 right_arm (431, 143, 3)
fx: 304, lx: 695, fy: 296, ly: 807
2 torso (511, 391, 3)
fx: 320, lx: 639, fy: 704, ly: 1623
3 left_leg (919, 319, 3)
fx: 304, lx: 327, fy: 696, ly: 1599
4 right_leg (903, 23, 3)
Saved images to output path rmbg_outputs/0.7
Created output path rmbg_outputs/0.7/10_29
fx: 624, lx: 727, fy: 328, ly: 599
0 left_arm (271, 103, 3)
fx: 336, lx: 431, fy: 328, ly: 591
1 right_arm (263, 95, 3)
fx: 392, lx: 679, fy: 272, ly: 727
2 torso (455, 287, 3)
fx: 520

In [ ]:
# fig, axes = plt.subplots(1, 5, figsize=(12, 16))
# images = [[]] * 5

# for i, part in enumerate(body_parts):
#   images[i] = Image.load_img(f'{output_path}/{part}.jpg')
#   image_array = Image.img_to_array(images[i])  
  
#   pixels_data = get_part_image(image_array=image_array)
  
#   axes[i].imshow(pixels_data)
#   axes[i].axis('off')
#   axes[i].set_title(f'{part}')
#   print(i, body_parts[part])
  
# plt.figure()
  